# Houses Price Prediction of Karachi

--------

## Import Libraries

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

--------

## Load Dataset

In [66]:
df = pd.read_csv('D:/DS Bootcamp/Machine Learning/Karachi House Price Prediction/Price-Prediction-of-Karachi-Houses/Karachi Houses Dataset.csv')
df.sample(5)

,property_id,purpose,city,location,beds,baths,area_yard,price,date_added
11008,Property_11009,Sale,Karachi,"Askari 5 - Sector H, Askari 5",5,5,427 Sq. Yd.,7.9 Crore,NaN
793,Property_794,Sale,Karachi,"Bahria Homes - Iqbal Villas, Bahria Town - Pre...",3,3,152 Sq. Yd.,19 Crore,2025-04-26
10448,Property_10449,Sale,Karachi,"Bukhari Commercial Area, DHA Phase 6",194 Sq. Yd.,NaN,NaN,2.4 Crore,2025-01-26
30,Property_31,Sale,Karachi,"North Nazimabad - Block N, North Nazimabad",7,7,256 Sq. Yd.,4 Crore,2025-04-26
5977,Property_5978,Sale,Karachi,"Bahria Town - Ali Block, Bahria Town - Precinc...",3,4,125 Sq. Yd.,1.5 Crore,2025-04-22


----------

## Data Preprocessing

In [67]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12025 entries, 0 to 12024
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  12025 non-null  object
 1   purpose      12025 non-null  object
 2   city         12025 non-null  object
 3   location     12025 non-null  object
 4   beds         12025 non-null  object
 5   baths        11226 non-null  object
 6   area_yard    11087 non-null  object
 7   price        11544 non-null  object
 8   date_added   6796 non-null   object
dtypes: object(9)
memory usage: 845.6+ KB


,property_id,purpose,city,location,beds,baths,area_yard,price,date_added
count,12025,12025,12025,12025,12025,11226,11087,11544,6796
unique,12025,1,1,637,84,49,314,637,20
top,Property_1,Sale,Karachi,"DHA Phase 6, DHA Defence",6,6,120 Sq. Yd.,6.5 Crore,2025-04-26
freq,1,12025,12025,797,2874,4132,2123,194,1334


### Data Anomilies 

Correct the data typos and anomilies on the dataset

From location extract only the house location area socities
like Scheme 33, DHA etc

In [68]:
import re
locations = ['DHA', 'Malir', 'Scheme 33', 'Naya Nazimabad', 'Gadap Town', 'North Nazimabad', 
             'Gulistan-e-Jauhar', 'Federal B Area', 'PECHS', 'Gulshan-e-Iqbal', 'Gulshan-e-Maymar', 
             'North Karachi', 'Shadman Town', 'Clifton', 'Askari 5', 'Surjani Town', 'Bahria', 'Scheme 1', 
             'Saima Villas', 'Saadi Garden', 'Karsaz', 'Saadi Town', 'Bufferzone', 'NHS Mauripur', 'KN Gohar Green', 
             'Gulshan-e-Roomi','Landhi','Falcon Complex', 'Saima Luxury', 'Jamshed Town','Amir Khusro', 'Ilyas Goth', 
             'Airport Road', 'Zamzama', 'Faisal Cantonment','Sindhi Muslim Society', 'Bath Island', 'Shahra-e-Faisal', 
             'Defence View', 'Shaheed Millat Road', 'Gulshan-e-Kaneez Fatima', 'Chapal Uptown']

def extract_location(location):
    for loc in locations:
        if loc in location:
            return loc
    return None

In [69]:
df['society'] = df['location'].apply(extract_location)

During scrapping the data scraped had some errors, like in baths and beds columns some values of area_yard were included

Correct the baths column 

In [70]:
area_condition = df['baths'].str.contains('Sq. Yd.', na=False)

df.loc[area_condition, 'area_yard'] = df.loc[area_condition, 'baths']

df.loc[area_condition, 'baths'] = None

Correct the beds column

In [71]:
area_condition = df['beds'].str.contains('Sq. Yd.', na=False)

df.loc[area_condition, 'area_yard'] = df.loc[area_condition, 'beds']

df.loc[area_condition, 'beds'] = None

Convert the area_yard into sq_feet

In [72]:
df['area_num'] = df['area_yard'].str.extract('(\d+)', expand=False).astype(float)

df['area_in_sqft'] = df['area_num'] * 9

Convert the price to numeric values

In [73]:
def convert_price(price):
    if isinstance(price, float):
        return price

    if 'Crore' in price:
        value = float(re.sub(r'[^\d.]', '', price))  
        return value * 10000000  
    elif 'Lakh' in price:
        value = float(re.sub(r'[^\d.]', '', price))  
        return value * 100000  
    
    return float(re.sub(r'[^\d.]', '', price))

In [74]:
df['numeric_price'] = df['price'].apply(convert_price)